# ObiTools

In [4]:
cd ../Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data

[Errno 2] No such file or directory: '../Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data'
/Users/tracykreling/Desktop/School/FISH546_Git/Sam-Metabarcoding/Data/Bat_Data


Check that your virtual environment is correctly finding Obitools

In [2]:
!which obi

/opt/miniconda3/envs/obitoolsenv/bin/obi


### Import the data

Note that the ngs filter must have the F in the 'extra' column or it will not work (thanks Sam for figuring that out!)

In [81]:
!head bat_ngs_atom.txt

#exp  sample  forward_primer  reverse_primer	extra_info
bat 2 AGATATTGGAACWTTATATTTTATTTTTGG  WACTAATCAATTWCCAAATCCTCC	F


In [6]:
!obi import --fastq-input 2_S2_L001_R1_001.fastq bat/reads1

Traceback (most recent call last):
  File "/opt/miniconda3/envs/obitoolsenv/bin/obi", line 44, in <module>
    from obitools3.apps.config import getConfiguration     # @UnresolvedImport
ImportError: dlopen(/opt/miniconda3/envs/obitoolsenv/lib/python3.9/site-packages/obitools3/apps/config.cpython-39-darwin.so, 2): Library not loaded: @rpath/libcobitools3.dylib
  Referenced from: /opt/miniconda3/envs/obitoolsenv/lib/python3.9/site-packages/obitools3/apps/config.cpython-39-darwin.so
  Reason: image not found


In [82]:
!obi import --fastq-input 2_S2_L001_R2_001.fastq bat/reads2

2021-03-01 12:16:13,496 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-03-01 12:16:13,509 [import : INFO ]  Opened file: 2_S2_L001_R2_001.fastq
2021-03-01 12:16:13,665 [import : INFO ]  Importing 64274 entries
 100.0 % |##################################################/] remain : 00:00:00
2021-03-01 12:16:16,254 [import : INFO ]  Imported 64274 entries
2021-03-01 12:16:16,423 [import : INFO ]  Done.


In [83]:
!obi import --ngsfilter bat_ngs_atom.txt bat/ngsfile

2021-03-01 12:16:16,784 [import : INFO ]  obi import: imports an object (file(s), obiview, taxonomy...) into a DMS
2021-03-01 12:16:16,788 [import : INFO ]  Opened file: bat_ngs_atom.txt
2021-03-01 12:16:16,818 [import : INFO ]  Importing 2 entries
  50.0 % |#########################-                         ] remain : 00:00:00
2021-03-01 12:16:16,856 [import : INFO ]  Imported 1 entries
2021-03-01 12:16:16,962 [import : INFO ]  Done.


### Check what is in the file structure

In [84]:
!obi ls bat

2021-03-01 12:16:19,017 [ls : INFO ]  obi ls
# DMS name: bat
# Views:
  # reads1: Date created: Mon Mar  1 12:15:55 2021 ; Line count: 64274
  # ngsfile: Date created: Mon Mar  1 12:16:16 2021 ; Line count: 1
  # reads2: Date created: Mon Mar  1 12:16:13 2021 ; Line count: 64274
# Taxonomies:



### Recover the full sequeunces from the partial foward and reverse reads

In [85]:
! obi alignpairedend -R bat/reads2 bat/reads1 bat/aligned_reads

2021-03-01 12:16:21,277 [alignpairedend : INFO ]  obi alignpairedend
 100.0 % |##################################################\] remain : 00:00:00
2021-03-01 12:16:29,699 [alignpairedend : INFO ]  Done.


### Remove unaligned sequence records

In [86]:
!obi grep -a mode:alignment bat/aligned_reads bat/good_sequences

2021-03-01 12:16:30,733 [grep : INFO ]  obi grep
 100.0 % |##################################################/] remain : 00:00:00
2021-03-01 12:16:31,096 [grep : INFO ]  Grepped 64267 entries
2021-03-01 12:16:31,150 [grep : INFO ]  Done.


### Export aligned reads

In [87]:
!obi alignpairedend -R bat/reads2 bat/reads1 - > aligned_reads.fastq

2021-03-01 12:16:34,385 [alignpairedend : INFO ]  obi alignpairedend
 100.0 % |##################################################\] remain : 00:00:00
2021-03-01 12:16:42,335 [alignpairedend : INFO ]  Printing to output...
 100.0 % |##################################################/] remain : 00:00:00
2021-03-01 12:16:45,278 [alignpairedend : INFO ]  Done.


### Assign each sequence record to the corresponding sample/marker combo

In [88]:
!obi ngsfilter -t bat/ngsfile -u bat/unidentified_sequences --no-tags bat/good_sequences bat/identified_sequences

2021-03-01 12:16:45,645 [ngsfilter : INFO ]  obi ngsfilter
 100.0 % |#################################################/ ] remain : 00:00:00
2021-03-01 12:17:14,196 [ngsfilter : INFO ]  Done.


### Dereplicate

In [3]:
!obi uniq -m sample bat/identified_sequences bat/dereplicated_sequences

Traceback (most recent call last):
  File "/opt/miniconda3/envs/obitoolsenv/bin/obi", line 44, in <module>
    from obitools3.apps.config import getConfiguration     # @UnresolvedImport
ImportError: dlopen(/opt/miniconda3/envs/obitoolsenv/lib/python3.9/site-packages/obitools3/apps/config.cpython-39-darwin.so, 2): Library not loaded: @rpath/libcobitools3.dylib
  Referenced from: /opt/miniconda3/envs/obitoolsenv/lib/python3.9/site-packages/obitools3/apps/config.cpython-39-darwin.so
  Reason: image not found


In [77]:
!obi --help

usage: obi [-h] [--version] [--log OBI:LOG]
           {addtaxids,align,alignpairedend,annotate,build_ref_db,cat,clean,clean_dms,count,ecopcr,ecotag,export,grep,head,history,import,less,ls,ngsfilter,sort,stats,tail,test,uniq}
           ...

optional arguments:
  -h, --help            show this help message and exit
  --version             Print the version of The OBITools 3
  --log OBI:LOG         Create a logfile

subcommands:
  valid subcommands

  {addtaxids,align,alignpairedend,annotate,build_ref_db,cat,clean,clean_dms,count,ecopcr,ecotag,export,grep,head,history,import,less,ls,ngsfilter,sort,stats,tail,test,uniq}
                        additional help
    addtaxids           Annotate sequences with their corresponding NCBI taxid
                        found from the taxon scientific name.
    align               Aligns one sequence column with itself or two sequence
                        columns
    alignpairedend      Aligns paired-ended reads
    annotate            Annotat

In [78]:
! obi clean_dms

error: the following arguments are required: INPUT
usage: obi clean_dms [-h] [--skip <N>] [--only <N>] INPUT

positional arguments:
  INPUT       Data source URI

optional arguments:
  -h, --help  show this help message and exit

Restriction to a sub-part options:
  Allows to limit analysis to a sub-part of the input

  --skip <N>  skip the N first sequences
  --only <N>  treat only N sequences
